In [1]:
import jn_setup
from simulation_procedure import model
from tools.paths import *
import pandas as pd, pyabc, hydroeval

To activate test mode, run 'inp_path = set_inp_path('test')'.
To activate run mode, run 'inp_path = set_inp_path('run')'.
C:\Users\jstelman\Git\stelman_urban_pesticides\master_test\NPlesantCreek.inp


In [2]:
"""
Priors. Get the values from that csv. Just for SWMM at first.
"""
swmm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

'''
Link up with the vvwm priors and make one big list with 36 params
'''
vvwm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges_vvwm.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

priors = param_ranges.to_dict("index")

# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})

In [5]:
prior

<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'Rough', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'rflat', 'sol', 'sused'>

# Make the .new object
### 1. Import observed data

In [3]:
# import it again to make inspect it
# specifically for TEST mode!
with open(os.path.join(main_path, 'master_test','test_obs_data.txt'),'r') as read_file:
    obs_dict = eval(read_file.read())
obs_dict

{'2009-02-13_26': 0.017,
 '2009-02-13_28': 0.0485,
 '2009-02-13_29': 0.203,
 '2009-04-07_28': 0.0192,
 '2009-04-07_29': 0.0985,
 '2009-04-08_26': 0.00774,
 '2009-04-13_26': 0.00088,
 '2009-04-13_28': 0.00858,
 '2009-04-13_29': 0.0257,
 '2009-05-01_26': 0.0088,
 '2009-05-01_28': 0.0145,
 '2009-05-01_29': 0.0203,
 '2009-08-28_26': 0.00088,
 '2009-08-28_28': 0.00553,
 '2009-08-28_29': 0.0168,
 '2009-10-12_26': 0.00088,
 '2009-10-12_28': 0.0074,
 '2009-10-12_29': 0.0141,
 '2009-10-13_26': 0.0195,
 '2009-10-13_28': 0.0513,
 '2009-10-13_29': 0.0332,
 '2010-04-04_26': 0.0114,
 '2010-04-04_28': 0.0388,
 '2010-04-04_29': 0.0405,
 '2010-07-19_26': 0.00088,
 '2010-07-19_28': 0.0146,
 '2010-07-19_29': 0.0343,
 '2010-07-19_42': 0.00088,
 '2010-10-24_26': 0.00088,
 '2010-10-24_29': 0.0317,
 '2010-10-24_35': 0.0094,
 '2010-10-24_38': 0.0067,
 '2010-10-24_42': 0.00088,
 '2011-02-16_26': 0.0799,
 '2011-02-16_29': 0.064,
 '2011-02-16_35': 0.0251,
 '2011-02-16_38': 0.0368,
 '2011-02-16_42': 0.00088}

### 2. Initialize dask client for dask distributed sampler

In [4]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()#n_workers=(90/2), threads_per_worker = 2)  # Set for 96 vCPU compute instance
client = Client(cluster)#,timeout=400)

sampler = pyabc.sampler.DaskDistributedSampler(dask_client = client)

### 3. Define ABCSMC object

In [10]:
abc = pyabc.ABCSMC(model, prior, distance_function = hydroeval.nse, 
                   population_size = pyabc.AdaptivePopulationSize(50, max_population_size = 55), # just to shorten the run
                   sampler = sampler)

In [25]:
abc

### 4. Set up a sqlite db directory

In [6]:
# Initialize a new ABC inference run
database_dir = os.path.join(temp_path, 'results_db')  
if not os.path.exists(database_dir):
    os.mkdir(database_dir)
db_path = ("sqlite:///" +
           os.path.join(database_dir, "run_11_18.db"))

### 5. Initialize a new abc run

In [11]:
abc.new(db_path, obs_dict)

In [12]:
history = abc.run(max_nr_populations=3, minimum_epsilon=0.2)

AttributeError: 'dict' object attribute 'values' is read-only

In [9]:
from simulation_procedure import model